In [1]:
import msprime
import toytree

ModuleNotFoundError: No module named 'msprime'

In [ ]:
def _get_demography(self):
        """
        Returns demography scenario based on an input tree and admixture
        edge list with events in the format (source, dest, start, end, rate).
        Time on the tree is defined in units of generations.
        """
        # Define demographic events for msprime
        demog = set()

        # tag min index child for each node, since at the time the node is
        # called it may already be renamed by its child index b/c of
        # divergence events.
        for node in self.tree.treenode.traverse():
            if node.children:
                node._schild = min([i.idx for i in node.get_descendants()])
            else:
                node._schild = node.idx

        # traverse tree from root to tips
        for node in self.tree.treenode.traverse():

            # if children add div events
            if node.children:
                dest = min([i._schild for i in node.children])
                source = max([i._schild for i in node.children])
                time = node.height  # int(node.height)
                demog.add(ms.MassMigration(time, source, dest))

                # for all nodes set Ne changes
                demog.add(ms.PopulationParametersChange(
                    time,
                    initial_size=node.Ne,
                    population=dest),
                )

            # tips set populations sizes (popconfig seemings does this too,
            # but it didn't actually work for tips until I added this...
            else:
                time = node.height  # int(node.height)
                demog.add(ms.PopulationParametersChange(
                    time,
                    initial_size=node.Ne,
                    population=node.idx,
                ))

            # debugging helper
            if self._debug:
                print(
                    'div time:  {:>9}, {:>2} {:>2}, {:>2} {:>2}, Ne={}'
                    .format(
                        time, source, dest,
                        node.children[0].idx, node.children[1].idx,
                        node.Ne,
                        ),
                    file=sys.stderr,
                )

        # Add migration pulses
        if not self.admixture_type:
            for evt in range(self.aedges):

                # rate is prop. of population, time is prop. of edge
                rate = self.ms_migrate[evt]
                time = self.ms_migtime[evt]
                source, dest = self.admixture_edges[evt][:2]

                # rename nodes at time of admix in case diverge renamed them
                snode = self.tree.treenode.search_nodes(idx=source)[0]
                dnode = self.tree.treenode.search_nodes(idx=dest)[0]
                children = (snode._schild, dnode._schild)
                demog.add(
                    ms.MassMigration(time, children[0], children[1], rate))
                if self._debug:
                    print(
                        'mig pulse: {:>9}, {:>2} {:>2}, {:>2} {:>2}, rate={:.2f}'
                        .format(
                            time, 
                            "", "",  # node.children[0].idx, node.children[1].idx, 
                            snode.name, dnode.name, 
                            rate),
                        file=sys.stderr,
                    )

        # Add migration intervals
        else:
            for evt in range(self.aedges):
                rate = self.ms_migration[evt]['mrates']
                time = (self.ms_migration[evt]['mtimes']).astype(int)
                source, dest = self.admixture_edges[evt][:2]

                # rename nodes at time of admix in case diverg renamed them
                snode = self.tree.treenode.search_nodes(idx=source)[0]
                dnode = self.tree.treenode.search_nodes(idx=dest)[0]
                children = (snode._schild, dnode._schild)
                demog.add(ms.MigrationRateChange(time[0], rate, children))
                demog.add(ms.MigrationRateChange(time[1], 0, children))
                if self._debug:
                    print("mig interv: {}, {}, {}, {}, {:.3f}".format(
                        time[0], time[1], children[0], children[1], rate),
                        file=sys.stderr)

        # sort events by type (so that mass migrations come before pop size
        # changes) and time
        demog = sorted(list(demog), key=lambda x: x.type)
        demog = sorted(demog, key=lambda x: x.time)
        self.ms_demography = demog